In [ ]:
# LOADING MODEL AND APPLYING LORA WEIGHTS 

from peft import PeftModel
from transformers import GenerationConfig
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer


tokenizer = LlamaTokenizer.from_pretrained("wxjiao/alpaca-7b")

model = LlamaForCausalLM.from_pretrained(
    "wxjiao/alpaca-7b",
    load_in_8bit=True,
    device_map="auto",
)

#Make sure to adjust the path in order to point it to the folder containing the LORA finetune files.

model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/lora-alpaca-counseling-finished")


In [ ]:
#SINGLE TURN MODEL INFERENCE

## To try out the model change the instruction text from "I am feeling pretty down lately and am not really sure what to do." to anything you wish. 

PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
I am feeling pretty down lately and am not really sure what to do.
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))